In [ ]:
import os

# Root directory of the KITTI dataset
kitti_base_path = "/kaggle/input/kitti-dataset"

# Training images and labels
kitti_images_dir = os.path.join(kitti_base_path, "data_object_image_2/training/image_2")
kitti_labels_dir = os.path.join(kitti_base_path, "data_object_label_2/training/label_2")

# YOLO dataset directories
yolo_dataset_path = "yolo_dataset"
yolo_images_train = os.path.join(yolo_dataset_path, "images/train")
yolo_images_val = os.path.join(yolo_dataset_path, "images/val")
yolo_labels_train = os.path.join(yolo_dataset_path, "labels/train")
yolo_labels_val = os.path.join(yolo_dataset_path, "labels/val")

# Create directories
os.makedirs(yolo_images_train, exist_ok=True)
os.makedirs(yolo_images_val, exist_ok=True)
os.makedirs(yolo_labels_train, exist_ok=True)
os.makedirs(yolo_labels_val, exist_ok=True)

print("Directories set up correctly.")


In [ ]:
import shutil
import numpy as np
from tqdm import tqdm

# KITTI class labels
kitti_classes = ["Car", "Pedestrian", "Cyclist", "Truck", "Van"]

def convert_kitti_to_yolo(label_file, split="train"):
    with open(os.path.join(kitti_labels_dir, label_file), "r") as f:
        lines = f.readlines()
    
    new_lines = []
    for line in lines:
        parts = line.strip().split()
        obj_class = parts[0]
        if obj_class in kitti_classes:
            x_min, y_min, x_max, y_max = map(float, parts[4:8])
            width = 1242  # Adjust if your image size differs
            height = 375

            x_center = ((x_min + x_max) / 2) / width
            y_center = ((y_min + y_max) / 2) / height
            bbox_width = (x_max - x_min) / width
            bbox_height = (y_max - y_min) / height

            yolo_class = kitti_classes.index(obj_class)
            new_lines.append(f"{yolo_class} {x_center} {y_center} {bbox_width} {bbox_height}\n")

    yolo_label_path = yolo_labels_train if split == "train" else yolo_labels_val
    with open(os.path.join(yolo_label_path, label_file), "w") as f:
        f.writelines(new_lines)

# Split dataset (80% train, 20% val)
image_files = sorted(os.listdir(kitti_images_dir))
split_idx = int(0.8 * len(image_files))

train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

# Process and move files
for file in tqdm(train_files, desc="Processing Training Data"):
    convert_kitti_to_yolo(file.replace('.png', '.txt'), "train")
    shutil.copy(os.path.join(kitti_images_dir, file), os.path.join(yolo_images_train, file))

for file in tqdm(val_files, desc="Processing Validation Data"):
    convert_kitti_to_yolo(file.replace('.png', '.txt'), "val")
    shutil.copy(os.path.join(kitti_images_dir, file), os.path.join(yolo_images_val, file))

print("KITTI annotations converted and dataset split.")


In [ ]:
dataset_yaml = f"""\
path: /kaggle/working/yolo_dataset
train: images/train
val: images/val
names:
  0: Car
  1: Pedestrian
  2: Cyclist
  3: Truck
  4: Van
"""

with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(dataset_yaml)

print("Updated dataset.yaml file created ✅")

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8s.pt")  # "yolov8s.pt" for a larger model

# Train the model
model.train(data="dataset.yaml", epochs=5, imgsz=640)

In [ ]:
# Validate trained model
model.val()

# Test on a sample validation image
test_image = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000001.png"  # Replace with an actual image path

results = model(test_image)  # Returns a list of results
results[0].show()  # Show the first result

In [ ]:
# Validate the model on the validation set
metrics = model.val()

# Extract mAP (Mean Average Precision)
mAP_50 = metrics.box.map50  # mAP at IoU=0.5
mAP_50_95 = metrics.box.map  # mAP at IoU=0.5:0.95

print(f"Model Accuracy (mAP@50): {mAP_50:.2f}")
print(f"Model Accuracy (mAP@50-95): {mAP_50_95:.2f}")